In [1]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd

In [2]:
!pip install splinter

You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!which chromedriver

/usr/local/bin/chromedriver


In [4]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
html = browser.html
soup_splinter = bs(html, 'html.parser')

In [6]:
name_string = ""
for i in range(0, len(asm_member_list)):
    this_name = "'" + asm_member_list[i] + "', "
    name_string += this_name

NameError: name 'asm_member_list' is not defined

In [8]:
#Dataframe with info on legislators
ca_asm_data = pd.DataFrame(pd.read_excel("Ca-Asm-Basic-Info.xlsx"))

In [9]:
#Creating a column for legislator last names
ca_asm_data["Last Name"] = "hi"
for i in range(0, len(ca_asm_data["Name"])):
    names = ca_asm_data["Name"][i].split()
    ca_asm_data["Last Name"][i] = names[-1]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [12]:
ca_asm_data["Ayes"] = 0
ca_asm_data["Nays"] = 0
ca_asm_data["Abstentions"] = 0

In [22]:
ca_asm_data.head()

,District,Name,Party,Residence,First elected,Term limited,Notes,Primary %,General %,Last Name,Ayes,Nays,Abstentions
0,1,Brian Dahle,Republican,Bieber,2012,2024,NaN,64.1,63.0,Dahle,1,3,5
1,2,Jim Wood,Democratic,Healdsburg,2014,2026,NaN,69.6,69.4,Wood,8,0,0
2,3,James Gallagher,Republican,Nicolaus,2014,2026,NaN,65.0,60.2,Gallagher,2,2,4
3,4,Cecilia Aguiar-Curry,Democratic,Winters,2016,2028,NaN,99.5,75.2,Aguiar-Curry,8,0,0
4,5,Frank Bigelow,Republican,O'Neals,2012,2024,NaN,59.9,62.4,Bigelow,4,0,0


In [11]:
#Gathering list of Assemblymember last names
asm_member_list = list(ca_asm_data["Last Name"])
#asm_member_list_a = []
#for i in range(0, len(asm_member_list)):
#    asm_member_list_a.append(asm_member_list[i] + " (A)")

In [9]:
"""
name_list = [" Wicks", " Bonta"]
name_list1 = ','.join(name_list)
df = pd.DataFrame(columns=['Bill', 'Sponsor', 'Dahle', 'Wood', 'Gallagher', 
                           'Aguiar-Curry', 'Bigelow', 'Kiley', 'McCarty', 'Cooley', 
                           'Cooper', 'Levine', 'Frazier', 'Flora', 'Eggman', 'Grayson', 
                           'Wicks', 'Bauer-Kahan', 'Chiu', 'Bonta', 'Ting', 
                           'Quirk', 'Gray', 'Mullin', 'Patterson', 'Berman', 'Chu', 'Mathis', 'Kalra', 'Low', 'Stone', 'Rivas', 'Arambula', 'Salas', 'Obernolte', 'Fong', 'Cunningham', 'Lackey', 'Limón', 'Smith', 'Rivas', 'Ramos', 'Holden', 'Mayes', 'Friedman', 'Irwin', 'Gabriel', 'Nazarian', 'Reyes', 'Rubio', 'Chau', 'Bloom', 'Carrillo', 'Rodriguez', 'Santiago', 'Kamlager', 'Chen', 'Garcia', 'Calderon', 'Garcia', 'Jones-Sawyer', 'Cervantes', 'Medina', 'Burke', 'Rendon', 'Gipson', 'Quirk-Silva', 'Muratsuchi', 'Melendez', 'Choi', 'Daly', "O'Donnell", 'Voepel', 'Diep', 'Brough', 'Petrie-Norris', 
                           'Waldron', 'Horvath', 'Maienschein', 'Gloria', 'Weber', 'Gonzalez'])
"""

In [20]:
#This function scrapes the CA Legislative website and puts data about legislations number,
#sponsors and votes
def bill_scraper(pages):

#Generating url
    for i in range(1,int(pages)):
        votes = []
        url = f"https://leginfo.legislature.ca.gov/faces/billVotesClient.xhtml?bill_id=201920200AB" + str(i)
        bill = f"AB{i}"
#Scraping url
        requested = requests.get(url)
        soup = bs(requested.text, 'html.parser')
        spans = soup.find_all('span')
#Getting and scraping status url for bill sponsors
        status_url = f"https://leginfo.legislature.ca.gov/faces/billStatusClient.xhtml?bill_id=201920200AB" + str(i)
        status_requested = requests.get(status_url)
        status_soup = bs(status_requested.text, 'html.parser')
        sponsor = status_soup.find_all('span', id="leadAuthors")[0].text
#Grabbing and cleaning ayes, noes and abstentions
        if len(spans) > 37:
            ayes = str(spans[38])
            noes = str(spans[40])
            abstentions = str(spans[42])
            ayes_clean = ayes[7:-7]
            noes_clean = noes[7:-7]
            abstentions_clean = abstentions[7:-7]
#turning each category into a list
            all_ayes = ayes_clean.split(", ")
            all_noes = noes_clean.split(", ")
            all_abstentions = abstentions_clean.split(", ")
#putting the bill into a dataframe
            for name in asm_member_list:
                member_index = ca_asm_data.index[ca_asm_data["Last Name"] == name].tolist()[0]
                if name in all_ayes:
                    ca_asm_data.iloc[member_index, 10] += 1
                elif name in all_noes:
                    ca_asm_data.iloc[member_index, 11] += 1
                elif name in all_abstentions:
                    ca_asm_data.iloc[member_index, 12] += 1
                else: print("not in committee")

In [ ]:
#Gathering lots of data
bill_scraper(2000)

not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committ

not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committ

not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committ

not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committ

not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committ

...

not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committ

not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committ

not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committ

not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committ

not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committee
not in committ

In [12]:
#With our scraper getting vote data, now we're going to incorporate 
#data about the legislators and their district

In [13]:
#Creating a column for sponsor last names in the bills dataframe
df["Sponsor Last Names"] = df["Sponsor"].str.split(",")

In [14]:
#Creating columns for number of Democrat and Republican sponsors
df["Dem Sponsors"] = 0
df["Rep Sponsors"] = 0

In [18]:
#This for loop looks at the sponsor column from the main df and the party column from the 
#member info df and adds number of sponsors by party to the appropriate column
for i in range (1, len(df["Sponsor Last Names"])):
    try:
        for j in range(0, (len(df["Sponsor Last Names"][i]))):
            sponsor_names = df["Sponsor Last Names"][i][j][0:-5]
            if sponsor_names[0] == " ":
                sponsor_names = sponsor_names[1:]
            sponsor_index = ca_asm_data.index[ca_asm_data["Last Name"] == sponsor_names].tolist()[0]
            sponsor_party = ca_asm_data.iloc[sponsor_index, 2]
            if sponsor_party == "Democratic":
                df["Dem Sponsors"][i] +=1
            elif sponsor_party == "Republican":
                df["Rep Sponsors"][i] +=1
            else: 
                print("no party")
    except:
        print("no bill")


no bill


In [27]:
df["Dem Sponsors"][1]

0

In [26]:
df.head()

,Bill,Sponsor,Dahle,Wood,Gallagher,Aguiar-Curry,Bigelow,Kiley,McCarty,Cooley,...,Petrie-Norris,Waldron,Horvath,Maienschein,Gloria,Weber,Gonzalez,Sponsor Last Names,Dem Sponsors,Rep Sponsors
1,AB1,Cooper (A),2,0,0,0,0,2,2,0,...,0,0,3,0,0,0,2,[Cooper (A) ],0,0
2,AB2,"Santiago (A) , Bonta (A) , Cervantes (A) , Chi...",2,0,2,0,0,2,0,2,...,0,0,3,0,0,0,0,"[Santiago (A) , Bonta (A) , Cervantes (A) , ...",0,0
3,AB3,Cooper (A),0,3,3,3,3,3,0,3,...,3,3,3,3,0,3,3,[Cooper (A) ],0,0
4,AB4,"Arambula (A) , Bonta (A) , Chiu (A) , Gonzalez...",1,0,1,0,3,1,0,2,...,2,1,3,2,0,0,0,"[Arambula (A) , Bonta (A) , Chiu (A) , Gonz...",0,0
5,AB5,Gonzalez (A),1,0,1,0,3,1,0,3,...,0,1,3,0,0,0,0,[Gonzalez (A) ],0,0


In [28]:
#Now to use machine learning to predict votes
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [29]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [30]:
list(le.classes_)

AttributeError: 'LabelEncoder' object has no attribute 'classes_'

In [24]:
encoded_asm_members = le.transform(asm_member_list_a)

NotFittedError: This LabelEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [31]:
X = df[["Dem Sponsors", "Rep Sponsors"]]
y = df["Wicks"]

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [60]:
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [63]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.7406716417910447
Testing Data Score: 0.7430167597765364


In [64]:
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
124,0,4
446,0,0
143,0,4
618,0,0
109,0,0
89,0,0
546,0,0
30,0,0
245,0,0
553,4,4
